In [111]:
import pandas as pd
from pprint import pprint as pprint
import numpy as np
from sklearn.cluster import AffinityPropagation, KMeans
import distance
import benepar, spacy
from transformers import pipeline
import templates
benepar.download('benepar_en3')

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/kostadindev/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [59]:

def get_phrases(sent, recursive=False):
    children = list(sent._.children)
    phrases = {}
    for i in range(len(children)):
        if str(children[i]) not in [',', ':', ';', '.', '!', '?']:
            try:
                label = children[i]._.labels[0]
            except:
                label = "stop"
            # print(children[i], label)
            # print(children[i], children[i]._.labels)
            # generate_questions(children[i])
            if label != 'stop':
                if label not in phrases.keys():
                    phrases[label] = []
                phrases[label].append(children[i])
                if recursive:
                    new_phrases = get_phrases(children[i], recursive=recursive)
                    for key in new_phrases.keys():
                        if key not in phrases.keys():
                            phrases[key] = []
                        phrases[key].extend(new_phrases[key])
    return phrases

In [99]:
def question(phrases):
    if 'SBAR' in phrases.keys():
        return f'What {phrases["SBAR"][0]}?'
    elif 'VP' in phrases.keys():
        return f'What {phrases["VP"][0]}?'

In [135]:
def answer(question, context, phrases, template):
    ans = pipeline("question-answering")
    ans = ans(question=question, context=context)
    #print(ans['score'])
    answers = []
    if template == "IF" or template =='THEN':
        if 'NP' in phrases.keys():
            for np in phrases['NP']:
                answers.append(f'{np}')
    elif template == "IF_THEN":
        if 'NP' in phrases.keys():
            for np in phrases['NP']:
                if 'VP' in phrases.keys():
                    for vp in phrases['VP']:
                        answers.append(f'{np} {vp}')
    return ans['answer'], answers

#%

In [126]:
sentence = 'If the stored verifier does not match, then an error of NFS4ERR_EXIST is returned.'

doc = nlp(sentence)
sent = list(doc.sents)[0]

/Users/kostadindev/.conda/envs/rfc-qa-generator/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [127]:
print("STEP 1: Get 1 layer constituency parse and determine template")
print(sentence)
phrases1 = get_phrases(sent, recursive= False)
pprint(phrases1)
print()
print("Determine it is template [IF THEN] from existance SBAR and ADVP")

STEP 1: Get 1 layer constituency parse and determine template
If the stored verifier does not match, then an error of NFS4ERR_EXIST is returned.
{'ADVP': [then],
 'NP': [an error of NFS4ERR_EXIST],
 'SBAR': [If the stored verifier does not match],
 'VP': [is returned]}

Determine it is template [IF THEN] from existance SBAR and ADVP


In [128]:
print("STEP 2: Ask question using SBAR and answer using QA")
print(f's: {sentence}')
q1 = question(phrases1)
print(f'q: {q1}')
ans1, answers1 = answer(q1, sentence, phrases1,"IF_THEN") # ans is deep learning answer and answers1 are the (NP,VP) pairs
print(f'possible answers: {answers1}')
print(f'deep learning answer: {ans1}')
print("select answer from possible answers using the deep learning answer")
print("Establish edge from this relationship")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


STEP 2: Ask question using SBAR and answer using QA
s: If the stored verifier does not match, then an error of NFS4ERR_EXIST is returned.
q: What If the stored verifier does not match?
possible answers: ['an error of NFS4ERR_EXIST is returned']
deep learning answer: an error of NFS4ERR_EXIST
select answer from possible answers using the deep learning answer
Establish edge from this relationship


In [129]:
print("STEP 3: Repeat step 1 and 2 for each subtree")
print("STEP 3.1: IF template")
sentence2_1 = phrases1['SBAR'][0]
print(f's: {sentence2_1}')

phrases2_1 = get_phrases(get_phrases(sentence2_1)['S'][0])

pprint(phrases2_1)

q2_1 = question(phrases2_1)
a2_1 = answer(q2_1,sentence, phrases2_1, template = 'IF' )
print(f'q: {q2_1} - pred_name')
print(f'a: {a2_1} - var_val')
print("if - if_else")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


STEP 3: Repeat step 1 and 2 for each subtree
STEP 3.1: IF template
s: If the stored verifier does not match
{'NP': [the stored verifier], 'VP': [does not match]}
q: What does not match? - pred_name
a: ('stored verifier', ['the stored verifier']) - var_val
if - if_else


In [137]:
print("STEP 3.2: THEN template")
sentence2_2 = list(nlp('an error of NFS4ERR_EXIST is returned').sents)[0]
print(f's: {sentence2_2}')

phrases2_2 = get_phrases(sentence2_2)
pprint(phrases2_2)
q2_2 = question(phrases2_2)
a2_2 = answer(q2_1,sentence, phrases2_1, template = 'THEN' )
print(f'q: {q2_2} - Def_Fun_name')
print(f'a: {a2_2} - return value')

/Users/kostadindev/.conda/envs/rfc-qa-generator/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


STEP 3.2: THEN template
s: an error of NFS4ERR_EXIST is returned
{'NP': [an error of NFS4ERR_EXIST], 'VP': [is returned]}
q: What is returned? - pred_name
a: ('stored verifier', ['the stored verifier']) - var_val
